# FRANKFURTER API - Currency Time Series Prediction

# Notebook 2 : Actualización

Documentación:https://www.frankfurter.app/docs/

The Frankfurter API tracks foreign exchange references rates published by the European Central Bank. The data refreshes around 16:00 CET every working day.

In [8]:
import pandas as pd
import numpy as np

import time
import datetime

import requests

from pprint import pprint

## Variables Globales

Variables necesarias para la extracción y organización de la información que nos brinda la API de Frankfurter.

In [9]:
url = "https://api.frankfurter.app"

## Funciones

Función para actualizar el DataFrame desde la última vez que recopiló la información hasta el día actual en el que se realice la actualización.

In [10]:
def update_dataframe(dataframe):
    
    from_currency = dataframe.columns[-2]
    init_date = dataframe["Dates"].iloc[-1]
    fin_date = datetime.datetime.today()
    amount = dataframe.iloc[-1, -2]
    
    days_dif = datetime.datetime.strptime(datetime.datetime.strftime(fin_date, "%Y-%m-%d"), "%Y-%m-%d") - datetime.datetime.strptime(init_date, "%Y-%m-%d")
    
    days_dif = days_dif.days
        
    if days_dif == 0 or (days_dif == 1 & fin_date.hour < 17):
        
        print("No hay actualizaciones disponibles.")
        print("La información se actualiza diariamente después de las 16:00 (hora estándar de Europa Central).")
        print("Inténtelo después.\n")
    
        return dataframe

    for days in range(1, days_dif + 1):

        updated_values = dict()

        date = datetime.datetime.strptime(init_date, "%Y-%m-%d") + datetime.timedelta(days = days)

        date = datetime.datetime.strftime(date, "%Y-%m-%d")

        endpoint = f"{url}/{date}?amount={amount}&from={from_currency}"

        response = requests.get(endpoint).json()

        for col in dataframe.columns[:-2]:

            try:

                updated_values[col] = response["rates"][col]

            except:

                updated_values[col] = np.nan

        updated_values["Dates"] = date

        updated_values[from_currency] = amount

        # Convertir el diccionario en un DataFrame
        updated_values_df = pd.DataFrame.from_dict(updated_values, orient = 'index', columns = [dataframe.shape[0]])

        # Concatenar el nuevo DataFrame al existente (axis=0 para apilar en filas)
        dataframe = pd.concat([dataframe, updated_values_df.T], axis = 0, sort = False)

        print(f"-----------{date} completed-----------")

    return dataframe
        
# VALIDAR LA CONDICIÓN DE SI SE REPITEN LOS MISMOS DATOS ENTONCES NO HAY ACTUALIZACIÓN YA QUE AUNQUE SEAN MÁS DE LAS 16:00 PUEDE QUE NO SE HAYA ACTUALIZADO TODAVÍA

## Actualización del DataFrame original

Cargamos el último DataFrame guardado para actualizarlo, si corresponde.

In [11]:
historical = pd.read_parquet("Data/historical.parquet")

In [12]:
historical

,AUD,BGN,BRL,CAD,CHF,CNY,CYP,CZK,DKK,EEK,...,SGD,SIT,SKK,THB,TRL,TRY,USD,ZAR,EUR,Dates
0,1.9100,NaN,NaN,1.8004,1.6168,NaN,0.58231,35.107,7.4501,15.6466,...,1.9554,189.04,42.991,NaN,372274.0,NaN,1.1789,6.9358,1,1999-01-04
1,1.8944,NaN,NaN,1.7965,1.6123,NaN,0.58230,34.917,7.4495,15.6466,...,1.9655,188.78,42.848,NaN,372816.0,NaN,1.1790,6.7975,1,1999-01-05
2,1.8820,NaN,NaN,1.7711,1.6116,NaN,0.58200,34.850,7.4452,15.6466,...,1.9699,188.70,42.778,NaN,372188.0,NaN,1.1743,6.7307,1,1999-01-06
3,1.8474,NaN,NaN,1.7602,1.6165,NaN,0.58187,34.886,7.4431,15.6466,...,1.9436,188.80,42.765,NaN,370147.0,NaN,1.1632,6.8283,1,1999-01-07
4,1.8406,NaN,NaN,1.7643,1.6138,NaN,0.58187,34.938,7.4433,15.6466,...,1.9537,188.84,42.560,NaN,371830.0,NaN,1.1659,6.7855,1,1999-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9030,1.6544,1.9558,5.2516,1.4311,0.9676,7.7728,NaN,24.377,7.4571,NaN,...,1.4524,NaN,NaN,38.396,NaN,28.931,1.0633,19.9410,1,2023-09-25
9031,1.6539,1.9558,5.2721,1.4318,0.9675,7.7517,NaN,24.411,7.4575,NaN,...,1.4500,NaN,NaN,38.549,NaN,28.919,1.0605,20.1390,1,2023-09-26
9032,1.6541,1.9558,5.2716,1.4255,0.9680,7.7031,NaN,24.470,7.4566,NaN,...,1.4449,NaN,NaN,38.541,NaN,28.767,1.0536,20.2220,1,2023-09-27
9033,1.6483,1.9558,5.3115,1.4206,0.9678,7.6986,NaN,24.378,7.4558,NaN,...,1.4431,NaN,NaN,38.657,NaN,28.895,1.0539,20.1700,1,2023-09-28


In [13]:
%%time

historical = update_dataframe(dataframe = historical)

-----------2023-09-30 completed-----------
-----------2023-10-01 completed-----------
-----------2023-10-02 completed-----------
-----------2023-10-03 completed-----------
-----------2023-10-04 completed-----------
-----------2023-10-05 completed-----------
-----------2023-10-06 completed-----------
-----------2023-10-07 completed-----------
-----------2023-10-08 completed-----------
-----------2023-10-09 completed-----------
-----------2023-10-10 completed-----------
-----------2023-10-11 completed-----------
-----------2023-10-12 completed-----------
-----------2023-10-13 completed-----------
-----------2023-10-14 completed-----------
-----------2023-10-15 completed-----------
-----------2023-10-16 completed-----------
-----------2023-10-17 completed-----------
-----------2023-10-18 completed-----------
CPU times: total: 719 ms
Wall time: 5.9 s


In [14]:
historical

,AUD,BGN,BRL,CAD,CHF,CNY,CYP,CZK,DKK,EEK,...,SGD,SIT,SKK,THB,TRL,TRY,USD,ZAR,EUR,Dates
0,1.91,NaN,NaN,1.8004,1.6168,NaN,0.58231,35.107,7.4501,15.6466,...,1.9554,189.04,42.991,NaN,372274.0,NaN,1.1789,6.9358,1,1999-01-04
1,1.8944,NaN,NaN,1.7965,1.6123,NaN,0.58230,34.917,7.4495,15.6466,...,1.9655,188.78,42.848,NaN,372816.0,NaN,1.179,6.7975,1,1999-01-05
2,1.882,NaN,NaN,1.7711,1.6116,NaN,0.58200,34.85,7.4452,15.6466,...,1.9699,188.70,42.778,NaN,372188.0,NaN,1.1743,6.7307,1,1999-01-06
3,1.8474,NaN,NaN,1.7602,1.6165,NaN,0.58187,34.886,7.4431,15.6466,...,1.9436,188.80,42.765,NaN,370147.0,NaN,1.1632,6.8283,1,1999-01-07
4,1.8406,NaN,NaN,1.7643,1.6138,NaN,0.58187,34.938,7.4433,15.6466,...,1.9537,188.84,42.560,NaN,371830.0,NaN,1.1659,6.7855,1,1999-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9049,1.6662,1.9558,5.3352,1.4397,0.9546,7.6902,NaN,24.67,7.4582,NaN,...,1.4405,NaN,NaN,38.255,NaN,29.258,1.0524,19.9616,1,2023-10-14
9050,1.6662,1.9558,5.3352,1.4397,0.9546,7.6902,NaN,24.67,7.4582,NaN,...,1.4405,NaN,NaN,38.255,NaN,29.258,1.0524,19.9616,1,2023-10-15
9051,1.666,1.9558,5.333,1.436,0.9505,7.7058,NaN,24.664,7.4592,NaN,...,1.4427,NaN,NaN,38.221,NaN,29.381,1.0538,19.8069,1,2023-10-16
9052,1.6622,1.9558,5.3305,1.4411,0.9517,7.7288,NaN,24.603,7.461,NaN,...,1.4471,NaN,NaN,38.482,NaN,29.508,1.0569,19.9138,1,2023-10-17


Guardo el DataFrame actualizado para entrenar el modelo con los datos más actuales posibles.

In [15]:
historical.to_parquet("Data/historical.parquet")

In [16]:
del historical

In [17]:
################################################## Sigue en FORECASTING ###################################################